In [1]:
PATH_TO_DATA = 'ml-latest-small'

In [2]:
import os

import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm, tqdm_notebook

from implicit.als import AlternatingLeastSquares

In [3]:
np.random.seed(42)

# Подготовка данных.

In [4]:
ratings = pd.read_csv(os.path.join(PATH_TO_DATA, 'ratings.csv'))

In [5]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags = pd.read_csv(os.path.join(PATH_TO_DATA, 'tags.csv'))

In [7]:
tags.head(5)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [8]:
tags['movieId'].unique().shape

(1572,)

In [9]:
ratings['movieId'].unique().shape

(9724,)

In [10]:
# Для упрощения, оставляем только фильмы по которым есть тэги

In [11]:
ratings = ratings[ratings['movieId'].isin(tags['movieId'].unique())]

In [12]:
tags = tags[tags['movieId'].isin(ratings['movieId'].unique())]

In [13]:
ratings['movieId'].unique().shape

(1554,)

In [14]:
tags['movieId'].unique().shape

(1554,)

In [15]:
# Поменяем индексы фильмов, чтобы сделать его плотным

In [16]:
movieIdToIndex = dict(zip(np.sort(ratings['movieId'].unique()), list(range(len(ratings['movieId'].unique())))))

In [17]:
IndexTomovieID = {v: k for (k, v) in movieIdToIndex.items()}

In [18]:
len(movieIdToIndex)

1554

In [19]:
ratings['movieId'] = ratings['movieId'].apply(lambda x: movieIdToIndex.get(x))

In [20]:
ratings['movieId'].unique().shape

(1554,)

In [21]:
tags['movieId'] = tags['movieId'].apply(lambda x: movieIdToIndex.get(x))

In [22]:
tags['movieId'].unique().shape

(1554,)

In [23]:
dict_of_titles = dict()
with open(os.path.join(PATH_TO_DATA, 'movies.csv')) as f:
    next(f)
    for line in f:
        line_splited = line.split(',')
        movieId = int(line_splited[0])
        title = line_splited[1]
        if movieId in movieIdToIndex:
            dict_of_titles[movieIdToIndex[movieId]] = title

In [24]:
# перемешаем все рейтинги
ratings = ratings.sample(frac=1)

In [25]:
# определим кол-во рейтингов в трейне
train_size = int(ratings.shape[0] * 0.8)

In [26]:
ratings_train = ratings.iloc[0: train_size].copy(deep=True)
ratings_test = ratings.iloc[train_size:].copy(deep=True)

In [27]:
print(ratings_train.shape)
print(ratings_test.shape)

(38629, 4)
(9658, 4)


In [28]:
tags.head(2)

,userId,movieId,tag,timestamp
0,2,1390,funny,1445714994
1,2,1390,Highly quotable,1445714996


С тэгами будем работать при помощи TF-IDF. Для этого представим надор тэгов для каждого фильма как предложение (тэги - слова, разделены пробелами)

In [29]:
tags['tag'] = tags['tag'].apply(lambda x: x.replace(' ', ''))

In [30]:
tags.head(2)

,userId,movieId,tag,timestamp
0,2,1390,funny,1445714994
1,2,1390,Highlyquotable,1445714996


In [31]:
tags = tags.groupby('movieId')['tag'].apply(list).reset_index()

In [32]:
tags.head()

,movieId,tag
0,0,"[pixar, pixar, fun]"
1,1,"[fantasy, magicboardgame, RobinWilliams, game]"
2,2,"[moldy, old]"
3,3,"[pregnancy, remake]"
4,4,[remake]


In [33]:
tags['tag'] = tags['tag'].apply(lambda x: ' '.join(x))

In [34]:
tags.head()

,movieId,tag
0,0,pixar pixar fun
1,1,fantasy magicboardgame RobinWilliams game
2,2,moldy old
3,3,pregnancy remake
4,4,remake


In [35]:
tags = tags.sort_values('movieId')

In [36]:
tags.head()

,movieId,tag
0,0,pixar pixar fun
1,1,fantasy magicboardgame RobinWilliams game
2,2,moldy old
3,3,pregnancy remake
4,4,remake


In [37]:
corpus = list(tags['tag'].values)

In [38]:
vectorizer = TfidfVectorizer()
items_tfidf_vectors = vectorizer.fit_transform(corpus)

In [39]:
items_tfidf_vectors.shape

(1554, 1502)

In [40]:
tag_sim_matrix = cosine_similarity(items_tfidf_vectors)

In [41]:
np.argsort(tag_sim_matrix[0])[-1: -4: -1]

array([  0, 543, 664], dtype=int64)

In [42]:
print(dict_of_titles[0])
print(dict_of_titles[543])
print(dict_of_titles[664])

Toy Story (1995)
"Bug's Life
Toy Story 2 (1999)


### Подготовим рекомендатель на основе схожести tfidf векторов тэгов. Используем ALS.

In [43]:
# 1502 размер векторов tf-idf
als_tags = AlternatingLeastSquares(factors=1502)

In [44]:
als_tags.item_factors = items_tfidf_vectors.toarray()

In [45]:
ratings_train.head()

,userId,movieId,rating,timestamp
66875,432,528,4.5,1316390945
96727,603,606,4.0,953926166
86158,560,205,4.0,1469657403
87772,566,118,4.0,849006808
88336,570,430,3.0,1181477324


In [46]:
# создаем csr матрицу с рейтингами
item_user_train = csr_matrix(
    (ratings_train['rating'].values, 
     (ratings_train['movieId'].values, ratings_train['userId'].values)),
    shape=(ratings_train['movieId'].max()+1, ratings_train['userId'].max()+1))

In [47]:
def compute_recommendations(u_id):
    recs = als_tags.recommend(
        userid=0,
        user_items=item_user_train.T[u_id],
        N=20,
        filter_already_liked_items=True,
        recalculate_user=True
    )
    
    return recs

In [48]:
compute_recommendations(140)

[(240, 1.0866290934551839),
 (810, 1.0780817015603277),
 (899, 1.0552020401381448),
 (715, 1.0433950269015073),
 (916, 1.0353881866080472),
 (660, 1.006274443941624),
 (896, 1.0000569567922084),
 (519, 0.9998015642518998),
 (298, 0.9997505618109784),
 (76, 0.9992086122479964),
 (333, 0.999000999000999),
 (325, 0.999000999000999),
 (287, 0.9984525878036319),
 (486, 0.9984023165991398),
 (36, 0.9983909172217559),
 (616, 0.9981850428170144),
 (993, 0.9981850428170144),
 (1, 0.9981707611199316),
 (293, 0.9980039920159681),
 (314, 0.9980039920159681)]

In [49]:
test_ratings = dict()
for uid in ratings_test['userId'].unique():
    test_ratings[uid] = set(ratings_test[ratings_test['userId'] == uid]['movieId'].values)

In [50]:
recommendations_by_tags = dict()
for uid in tqdm(list(test_ratings.keys())):
    recommendations_by_tags[uid] = compute_recommendations(uid)

100%|████████████████████████████████████████████████████████████████████████████████| 596/596 [26:35<00:00,  1.42it/s]


## Подготовим метрики для оценки качества рекомендаций.

In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score

### Задание 1.
Посчитать руками метрики и проверить свои результаты, используя функции из sklearn

In [52]:
predicted = np.array([1, 1, 1, 0, 0, 0])
actual = np.array([1, 1, 0, 0, 1, 1])

precision = ???

recall = ???

f-measure - среднее гармоническое между precision и recall

f-measure = ???

In [53]:
precision_score(actual, predicted)

0.6666666666666666

In [54]:
recall_score(actual, predicted)

0.5

In [55]:
f1_score(actual, predicted)

0.5714285714285715

### Задание 2.
Подготовить метрики precision_score, recall_score, f1_score так, чтобы их можнобыло применить к реклмендациям.

In [56]:
# делали на предыдущем занятии
def avg_precision_topK(test_ratings, recommendations, k=5):
    total_precision = list()
    for uid in recommendations:
        if uid in test_ratings:
            pred_uid = sorted(recommendations[uid], key=lambda x: -x[1])[0: k]
            user_precision = len(set([movie for (movie, score) in pred_uid]) & test_ratings[uid]) / k
            total_precision.append(user_precision)
    return np.mean(total_precision)

In [70]:
avg_precision_topK(test_ratings, recommendations_by_tags, k=5)

0.0023489932885906043

In [71]:
def avg_recall_topK(test_ratings, recommendations, k=5):
    total_recall = list()
    for uid in recommendations:
        if uid in test_ratings:
            pred_uid = sorted(recommendations[uid], key=lambda x: -x[1])[0: k]
            user_precision = len(set([movie for (movie, score) in pred_uid]) & test_ratings[uid]) / len(test_ratings[uid])
            total_recall.append(user_precision)
    return np.mean(total_recall)

In [72]:
avg_recall_topK(test_ratings, recommendations_by_tags, k=5)

0.0005101517471216172

In [62]:
def harmonic_mean(a, b):
    if (a+b) == 0: 
        return 0
    else:
        return 2*a*b/(a+b)

def avg_f1_topK(test_ratings, recommendations, k=5):
    total_f1 = list()
    for uid in recommendations:
        if uid in test_ratings:
            pred_uid = sorted(recommendations[uid], key=lambda x: -x[1])[0: k]
            user_precision = len(set([movie for (movie, score) in pred_uid]) & test_ratings[uid]) / k
            user_recall = len(set([movie for (movie, score) in pred_uid]) & test_ratings[uid]) / len(test_ratings[uid])
            total_f1.append(harmonic_mean(user_precision, user_recall))
    return np.mean(total_f1)

In [63]:
avg_f1_topK(test_ratings, recommendations_by_tags, k=5)

0.0006924162481604029

### Задание 3.
Подготовить рекомендательные метрики для оценки покрытия и разнообразия.

In [ ]:
# Покрытие. Какую часть каталога покрывают рекомендации.
def covarage(recommendations, number_of_unique_items):
    

In [ ]:
from itertools import combinations

In [ ]:
# Разнообразие рекомендаций. Можно считать как среднее попарное растояние между рекомендованными объектами
# чтобы перебрать пары айтемов в рекомендациях используем combinations
def diversity(recommendations, items_similarity_matrix):
    ???

### Задание 4.
Посчитать precision, recall, f1_score, coverage и diversity для рекомендаций на основе тэгов.

In [ ]:
ratings_train['movieId'].unique().shape

### Задание 5.
Сделать рекомендатель на основе рейтигов (стандартный ALS)

In [ ]:
# у ALS указываем только кол-во факторов: 20
als = ???

In [ ]:
als.fit(???)

In [ ]:
def compute_recommendations_main_als(u_id):
    # хотим получать топ 20 рекомендаций для каждого пользователя
    recs = als.recommend(???)
    
    return recs

In [ ]:
recommendations_by_main_als = dict()
for uid in tqdm(list(test_ratings.keys())):
    recommendations_by_main_als[uid] = compute_recommendations_main_als(uid)

In [ ]:
print(avg_precision_topK(test_ratings, recommendations_by_main_als))
print(avg_recall_topK(test_ratings, recommendations_by_main_als))
print(avg_f1_topK(test_ratings, recommendations_by_main_als))
print(covarage(recommendations_by_main_als, 1516))
print(diversity(recommendations_by_main_als, als_sim_matrix))

### Задание 6.

Сделать двух этапную модель рекомендаций.  
* Первый уровен: ALS;
* Второй уровень: GradientBoostingClassifier;

In [ ]:
ratings_train.head()

Нам нужно разделить train еще на 2 группы, так как, нужно будет обуучать 2 модели, одна из которых будет использовать выход другой модели

In [ ]:
# перемешаем все рейтинги в трейн
ratings_train = ratings_train.sample(frac=1)

In [ ]:
# определим кол-во рейтингов в новом трейне
train_size = int(ratings_train.shape[0] * 0.8)

In [ ]:
ratings_train_train = ratings_train.iloc[0: train_size].copy(deep=True)
ratings_train_valid = ratings_train.iloc[train_size:].copy(deep=True)

In [ ]:
print(ratings_train_train.shape)
print(ratings_train_valid.shape)

In [ ]:
# создаем csr матрицу с рейтингами
# используем ratings_train_train
item_user_train = ???

In [ ]:
# у ALS указываем только кол-во факторов: 20
als = ???

In [ ]:
als.fit(item_user_train)

In [ ]:
# возвращаем топ 500 рекомендаций, нужно убедиться, что этого достаточно
def compute_recommendations_main_als(u_id):
    recs = als.recommend(???)
    
    return recs

In [ ]:
recommendations_by_main_als = dict()
for uid in tqdm(list(test_ratings.keys())):
    recommendations_by_main_als[uid] = compute_recommendations_main_als(uid)

In [ ]:
# # какую метрику нужно сосчитать, чтобы убедится что такого кол-ва рекомендаций достаточно
???

In [ ]:
# функция будет возвращать датафрейм с кандидатами для модели 2ого уровня
def create_candidates_dataframe_with_scores(user_ids):
    # здесь будем собирать колонку с user_id
    user_id_column = []
    # здесь будем собирать колонку с item_id
    item_id_column = []
    # здесь будем собирать колонку с als score
    als_score_column = []
    
    for uid in tqdm(user_ids):
        items_with_scores = compute_recommendations_main_als(uid)
        item_ids, scores = zip(*items_with_scores)
        ???
        
    df = pd.DataFrame({
        'user_id': user_id_column,
        'item_id': item_id_column,
        'als_score': als_score_column
    })
    return df

In [ ]:
valid_user_ids = ratings_train_valid['userId'].unique()
df_train_valid = create_candidates_dataframe_with_scores(valid_user_ids)

In [ ]:
df_train_valid.head()

In [ ]:
# добавим таргет (был рейтинг или нет?)

In [ ]:
df_train_valid.head()

In [ ]:
ratings_train_valid['rating'] = 1.0

In [ ]:
ratings_train_valid.head()

In [ ]:
# нужно сделать pd.merge двух датафреймов df_train_valid и df_train_valid
df_train_valid = pd.merge(df_train_valid, ???)[['user_id', 'item_id', 'als_score', 'rating']]

In [ ]:
# заполним нулями пропущеные значения столбца 'rating'
df_train_valid['rating'] = ???

In [ ]:
df_train_valid.head()

#### Какие фичи можно добавить?
* Популярность контента
* Кол-во оцененных фильмов пользователем
* Кол-во фильмов с похожими тэгами, которые пользователь уже посмотрел (попробовать дома)

In [ ]:
# добавим популярность контента

In [ ]:
ratings_train_train.head()

In [ ]:
# сделать groupby, чтобы найти поплярность фильма
movie_popularity = ratings_train_train.groupby ???
movie_popularity.columns = ['movieId', 'item_cnt']

In [ ]:
movie_popularity.head()

In [ ]:
# объединим df_train_valid и movie_popularity по item_id
df_train_valid = pd.merge(df_train_valid, ???)
df_train_valid = df_train_valid[['user_id', 'item_id', 'als_score', 'item_cnt', 'rating']]

In [ ]:
df_train_valid.head()

In [ ]:
# добавим кол-во оцененных фильмов пользователем фильмов

In [ ]:
# сделать groupby, чтобы найти кол-во оцененных фильмов пользователем фильмов
user_cnt = ratings_train_train.groupby ???
user_cnt.columns = ['userId', 'user_cnt']

In [ ]:
# объединим df_train_valid и user_cnt по user_id
df_train_valid = pd.merge(df_train_valid, ???)
df_train_valid = df_train_valid[['user_id', 'item_id', 'als_score', 'item_cnt', 'user_cnt', 'rating']]

In [ ]:
df_train_valid.head()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbc = GradientBoostingClassifier()

In [ ]:
# подготовить numpy array для обучения
X = ???
y = ???

In [ ]:
gbc.fit(X, y)

#### Подготовить датасет для предсказания рекомендаций.

In [ ]:
# используем create_candidates_dataframe_with_scores
test_user_ids = ratings_test['userId'].unique()
df_test = create_candidates_dataframe_with_scores(???)

In [ ]:
# добавим к данным movie_popularity и  user_cnt
df_test = ???
df_test = df_test[['user_id', 'item_id', 'als_score', 'item_cnt']]
df_test = ???
df_test = df_test[['user_id', 'item_id', 'als_score', 'item_cnt', 'user_cnt']]

In [ ]:
X_test = df_test[['als_score', 'item_cnt', 'user_cnt']].values

In [ ]:
gbc_score = gbc.predict_proba(X_test)

In [ ]:
df_test['score'] = gbc_score[:, 1]

In [ ]:
df_test.head()

In [ ]:
dict_of_prediction = {}
for uid in test_user_ids:
    user_scores = df_test[df_test['user_id'] == uid][['item_id', 'score']]
    dict_of_prediction[uid] = list(zip(user_scores['item_id'].values, user_scores['score'].values))

In [ ]:
print(avg_precision_topK(test_ratings, recommendations_by_main_als))
print(avg_recall_topK(test_ratings, recommendations_by_main_als))
print(avg_f1_topK(test_ratings, recommendations_by_main_als))
print(covarage(recommendations_by_main_als, 1516))
print(diversity(recommendations_by_main_als, als_sim_matrix))

In [ ]:
print(avg_precision_topK(test_ratings, dict_of_prediction))
print(avg_recall_topK(test_ratings, dict_of_prediction))
print(avg_f1_topK(test_ratings, dict_of_prediction))
print(covarage(dict_of_prediction, 1516))
print(diversity(dict_of_prediction, als_sim_matrix))